In [52]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
logs_df_statistical_method = pd.read_csv("/content/drive/MyDrive/logs_df_statistical_method.csv")
logs_df_statistical_method.head()

,Unnamed: 0,request_processing_time,target_processing_time,response_processing_time,received_bytes,sent_bytes,number of requests
0,0,0.000,0.759,0.0,375,542,1
1,1,0.001,0.194,0.0,374,2363,5
2,2,0.001,0.444,0.0,374,542,7
3,3,0.000,0.221,0.0,375,5641,8
4,4,0.001,0.168,0.0,375,2532,8


In [41]:
logs_df_statistical_method.drop('Unnamed: 0', axis=1, inplace=True)

In [42]:
logs_df_statistical_method.shape

(100000, 6)

Cutoff hi > 2 * (p + 1) / n was too weak as all observations were marked as unusual. We will use hi > 3 * (p + 1) / n cutoff instead.

In [43]:
# statistical model unidimensional case
anomalies_features = list()
anomalies_indices = list()
for column in logs_df_statistical_method.columns:
    for i in tqdm(range(1, logs_df_statistical_method.shape[0])):
        if i == 1:
          current_mean = np.mean(logs_df_statistical_method[column][:2])
        else:
          current_mean = ((i - 1) * previous_mean + logs_df_statistical_method[column][i]) * 1/i
        leverage_statistic = 1/i + np.power(logs_df_statistical_method[column][i] - current_mean, 2) / (np.var(logs_df_statistical_method[column][:i]) * 1/i)
        if leverage_statistic > (3 * 2) / i: # hi > 3 * (p + 1) / n
            anomalies_features.append(column)
            anomalies_indices.append(i)
        previous_mean = current_mean

  0%|          | 0/99999 [00:00<?, ?it/s]<ipython-input-43-b8314c851986>:10: RuntimeWarning: divide by zero encountered in double_scalars
  leverage_statistic = 1/i + np.power(logs_df_statistical_method[column][i] - current_mean, 2) / (np.var(logs_df_statistical_method[column][:i]) * 1/i)
  0%|          | 0/99999 [00:00<?, ?it/s]<ipython-input-43-b8314c851986>:10: RuntimeWarning: invalid value encountered in double_scalars
  leverage_statistic = 1/i + np.power(logs_df_statistical_method[column][i] - current_mean, 2) / (np.var(logs_df_statistical_method[column][:i]) * 1/i)
100%|██████████| 99999/99999 [00:50<00:00, 1978.70it/s]


In [44]:
# analyzing the output
len(set(anomalies_indices))

99998

In [45]:
# analyzing the output
np.unique(np.array(anomalies_features), return_counts=True)

(array(['number of requests', 'received_bytes', 'request_processing_time',
        'response_processing_time', 'sent_bytes', 'target_processing_time'],
       dtype='<U24'),
 array([99998, 99994, 99933, 99627, 99987, 99967]))

In [53]:
# statistical model multidimensional case
anomalies_indices_md = list()
leverage_statistics = list()
design_matrix_product_inverse = np.linalg.inv(np.dot(np.transpose(logs_df_statistical_method.to_numpy()), logs_df_statistical_method.to_numpy()))
for i in tqdm(range(1, logs_df_statistical_method.shape[0])):
  leverage_statistic = np.dot(np.dot(np.transpose(logs_df_statistical_method.iloc[0].to_numpy()), design_matrix_product_inverse), logs_df_statistical_method.iloc[0].to_numpy())
  leverage_statistics.append(leverage_statistic)
  if leverage_statistic > (3 * (7)) / i: # hi > 3 * (p + 1) / n
    anomalies_indices_md.append(i)

100%|██████████| 99999/99999 [00:23<00:00, 4192.68it/s]


In [54]:
# analyzing the output
len(anomalies_indices_md)

0

In [48]:
set(anomalies_indices).intersection(set(anomalies_indices_md))

set()

In [51]:
len(set(anomalies_indices).difference(set(anomalies_indices_md)))

99998